In [1]:
using JuMP, Plasmo, MPI

MPI.Init()

# Model parameters

NS = 5;                  # number of scenarios
S = collect(1:NS);       # scenario set
P = collect(1:3);        # set of crops (1=wheat,2=corn,3=beets)

# Data
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data
yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;
yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

# Create the Plasmo Graph Model
graph = GraphModel()
master = Model()
master_node = add_node(graph,master)

@variable(master, x[P] >= 0)    # acres devoted to crops
@constraint(master, cap, sum(x[j] for j in P) <= 500)

children_nodes = Array{NodeOrEdge}(NS)
for s = 1:NS
    q = Model()
    child = add_node(graph,q)
    children_nodes[s] = child
    @variable(q, y[P] >= 0)    # crops purchase
    @variable(q, w[P] >= 0)    # crops sold
    @variable(q, cost)         # per scenario cost
    @linkconstraint(graph, cost == sum(prcost[j]*master[:x][j] + pcost[j]*y[j] - scost[j]*w[j] for  j in P))
    @linkconstraint(graph, [j in P], yield[s,j]*master[:x][j]+y[j]-w[j] >= demand[j])
    @constraint(q, sellb, w[3] <= 6000)
    @constraint(q, buyb, y[3] <= 0)
    @objective(q, Min, cost)
end

# Dsp solve types
solve_types = [:Dual, :Benders]
status = dsp_solve(graph,master_node,children_nodes,solve_type = solve_types[2])  #probabilities are 1/NS by default

# Results
println(getvalue(x))
println("")

for node in children_nodes
    println(getvalue(node[:cost]))
end
println("")
println("obj ", getobjectivevalue(master))

MPI.Finalize();

Finding a good lower bound using Dual Decomposition...

DUAL DECOMPOSITION ITERATION INFORMATION:
* master   = objective function value of the master problem.
* primobj  = best primal objective function value.
* dualobj  = best dual objective function value.
* a.gap(%) = Approximate gap between master and dualobj.
* d.gap(%) = Duality gap between primobj and dualobj.
* times    = wall clock time in seconds.

  iter         master        primobj        dualobj  a.gap(%)  d.gap(%)    time
 D   0  -1.129800e+05          Large  -1.138600e+05      0.78    100.00     0.0
 D   1  -1.117773e+05          Large  -1.133200e+05      1.38    100.00     0.0
 D   2  -1.104169e+05          Large  -1.117661e+05      1.22    100.00     0.0
 D   3  -1.090516e+05          Large  -1.104106e+05      1.25    100.00     0.0
 D   4  -1.076911e+05          Large  -1.090508e+05      1.26    100.00     0.0
 D   5  -1.063309e+05          Large  -1.076908e+05      1.28    100.00     0.0
 D   6  -1.049709e+05       